In [18]:
!pip install --quiet ipynb

In [34]:
from typing import Callable
import numpy as np
import math
import inspect
import sys
from ipynb.fs.full.cubic import cubic

# Things to do / Ideas
1. Create functions to test
2. Calculate the error based on the degree of the Simpson's rule
3. Graph the error (and possibly a visualization of the functions)

In [3]:
def simps_2d(f: Callable[[float], float], a: float, b:float) -> float:
    m = (a+b)/2
    return ((b-a)/6)*(f(a) + 4*f(m) + f(b))

In [4]:
f = lambda x: x**2
simps_2d(f, -1, 1)

0.6666666666666666

In [9]:
def general_simpson(f: Callable[[float], float], i: int, a: float, b: float) -> float:
    xs = np.linspace(a, b, i, True, dtype = np.float64)
    A: list[np.float64] = []
    assert i >= 2
    for i in range(1, len(xs)):
        m = (xs[i] + xs[i-1])/2
        a = (1/6) * (f(xs[i]) + 4*f(m) + f(xs[i-1]))*(xs[i]-xs[i-1])
        A.append(a)
    return sum(A)

In [10]:
print(general_simpson(f, 2, -1, 1))
f3 = lambda x: x**3 + x**2 + x + 1
print(general_simpson(f3, 5, -1, 2))
f4 = lambda x: x**4
print(general_simpson(f4, 4, -1, 2))
sinfun = lambda x: math.sin(x) + x**3
print(general_simpson(sinfun, 2, 0, math.pi/2))
expfun = lambda x: math.exp(x)
print(general_simpson(expfun, 4, 0, 1))
print(math.e - 1)

0.6666666666666666
11.25
6.625
2.5242969248984983
1.7182891699208316
1.718281828459045


In [11]:
def format_func_str(input: str) -> str:
    return input.replace('    lambda x: ', '').replace('**', '^').replace(',', '').replace('math.', '').strip(' \n')

In [28]:
funcs = [
    lambda x: x + 1, 
    lambda x: x**2 + 1, 
    lambda x: x**3 + 1, 
    lambda x: x**4 + 1, 
    lambda x: x**5 + 1,
    lambda x: math.exp(x),
    lambda x: math.sin(x)
    ]

integrals = [
    lambda x: (x**2)/2 + x, 
    lambda x: (x**3)/3 + x, 
    lambda x: (x**4)/4 + x, 
    lambda x: (x**5)/5 + x, 
    lambda x: (x**6)/6 + x,
    lambda x: math.exp(x),
    lambda x: -math.cos(x)
    ]

methods = [
    ('Simpson', general_simpson),
    ('Cubic', cubic)
]

In [13]:
for fun, inte in zip(funcs, integrals):
    print("f(x) =", format_func_str(inspect.getsource(fun)))
    print("g(x) =", format_func_str(inspect.getsource(inte)))
    for i in range (2, 5):
        simp = general_simpson(fun, i, -1, 1)
        tru = inte(1)-inte(-1)
        diff = abs(tru-simp)
        if diff > 1e-10:
            print(f'dx: {i}\nTrue value: {tru}\nObtained value: {simp}\nDifference: {diff}\n')


f(x) = x + 1
g(x) = (x^2)/2 + x
f(x) = x^2 + 1
g(x) = (x^3)/3 + x
f(x) = x^3 + 1
g(x) = (x^4)/4 + x
f(x) = x^4 + 1
g(x) = (x^5)/5 + x
dx: 2
True value: 2.4
Obtained value: 2.6666666666666665
Difference: 0.2666666666666666

dx: 3
True value: 2.4
Obtained value: 2.4166666666666665
Difference: 0.016666666666666607

dx: 4
True value: 2.4
Obtained value: 2.4032921810699586
Difference: 0.003292181069958655

f(x) = x^5 + 1
g(x) = (x^6)/6 + x
f(x) = exp(x)
g(x) = exp(x)
dx: 2
True value: 2.3504023872876028
Obtained value: 2.362053756543496
Difference: 0.011651369255893052

dx: 3
True value: 2.3504023872876028
Obtained value: 2.3511948318802554
Difference: 0.0007924445926525969

dx: 4
True value: 2.3504023872876028
Obtained value: 2.350561486811035
Difference: 0.00015909952343218237

f(x) = sin(x)
g(x) = -cos(x)


## General idea for higher order integrations
n -> degree

newton-cotes formula

$$f(a) + \sum_{i=1}^{n-1}(k_{i}f(\frac{i(a+b)}{n}) + f(b)$$

# What we want to do:
1. Figure out the higher oder formulas
    1. Show how we get the formulas
2. Compare them (which one would require fewer dx to get close to actual integration)
    1. Also compare to quadrature shown in class
3. Graph up the results?

In [35]:
def masterloop(functions: list, integrals: list, methods, a: float, b: float):
    for func, inte in zip(functions, integrals):
        print(f"Function: {format_func_str(inspect.getsource(func))}, Integral: {format_func_str(inspect.getsource(inte))}")
        for name, meth in methods:
            print(name, '\n')
            for i in range(2, 10):
                # Get the method result, compare to actual integral result, get error (if meaningful)
                # method has 4 parameters: function, partitions, a, b
                r = meth(func, i, a, b)
                print("Result:",r)
                expected = (inte(b)-inte(a))
                print("Actual:", expected)
                print("Error:", abs(expected- r) if abs(expected-r) > sys.float_info.epsilon else 0)

In [36]:
masterloop(funcs, integrals, methods, -1, 1)


Function: x + 1, Integral: (x^2)/2 + x
Simpson 

Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 1.9999999999999998
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Cubic 

Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Result: 2.0
Actual: 2.0
Error: 0
Function: x^2 + 1, Integral: (x^3)/3 + x
Simpson 

Result: 2.6666666666666665
Actual: 2.6666666666666665
Error: 0
Result: 2.6666666666666665
Actual: 2.6666666666666665
Error: 0
Result: 2.6666666666666665
Actual: 2.6666666666666665
Error: 0
Result: 2.6666666666666665
Actual: 2.6666666666666665
Error: 0
Result: 2.6666666666666665
Actual: 2.6666666666666665
Error: 0
Result: 2.6666666666666665
Actual: